## Import useful libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import LabelEncoder

seed = 42

## Load the data and visualize it

In [2]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
print(f"Training set: \t{data.shape[0]} rows \t{data.shape[1]} columns")
print(f"Test set: \t{data_test.shape[0]} rows \t{data_test.shape[1]} columns")

Training set: 	188533 rows 	13 columns
Test set: 	125690 rows 	12 columns


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [6]:
data.describe(include='object')

,brand,model,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
count,188533,188533,183450,188533,188533,188533,188533,186081,167114
unique,57,1897,7,1117,52,319,156,2,1
top,Ford,F-150 XLT,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes
freq,23088,2945,165940,3462,49904,48658,107674,144514,167114


In [7]:
data.isnull().sum()

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

Replace '-' with 'Electric' because electric is mentioned in the "Data card" of the dataset and it is not in the values, so it must be the '-'

In [8]:
print('Fuel types pre processing:', data['fuel_type'].unique())

data['fuel_type'] = data['fuel_type'].fillna('Unknown')
data['fuel_type'] = data['fuel_type'].replace('–', 'Electric')

print('Fuel types post processing:', data['fuel_type'].unique())

data_test['fuel_type'] = data_test['fuel_type'].fillna('Unknown')
data_test['fuel_type'] = data_test['fuel_type'].replace('–', 'Electric')

Fuel types pre processing: ['Gasoline' 'E85 Flex Fuel' nan 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–'
 'not supported']
Fuel types post processing: ['Gasoline' 'E85 Flex Fuel' 'Unknown' 'Hybrid' 'Diesel' 'Plug-In Hybrid'
 'Electric' 'not supported']


In [9]:
print('Accidents pre processing:', data['accident'].unique())

data['accident'] = data['accident'].fillna('Unknown')

print('Accidents post processing:', data['accident'].unique())

data_test['accident'] = data_test['accident'].fillna('Unknown')


Accidents pre processing: ['None reported' 'At least 1 accident or damage reported' nan]
Accidents post processing: ['None reported' 'At least 1 accident or damage reported' 'Unknown']


In [10]:
print('Clean title pre processing:', data['clean_title'].unique())

data['clean_title'] = data['clean_title'].fillna('Unknown')

print('Clean title post processing:', data['clean_title'].unique())

data_test['clean_title'] = data_test['clean_title'].fillna('Unknown')

Clean title pre processing: ['Yes' nan]
Clean title post processing: ['Yes' 'Unknown']


In [11]:
print('TRAIN SET:')
print(data.isnull().sum())
print('\nTEST SET:')
print(data_test.isnull().sum())

TRAIN SET:
id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

TEST SET:
id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64


In [12]:
label_columns = ['brand', 'model', 'fuel_type', 'ext_col', 'int_col', 'accident', 'clean_title']
encoders = {col: LabelEncoder() for col in label_columns}

for col in label_columns:
    data[col] = encoders[col].fit_transform(data[col])
    data_test[col] = encoders[col].fit_transform(data_test[col])

data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,31,495,2007,213000,3,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,312,71,1,1,4200
1,1,28,930,2002,143250,3,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,263,10,0,1,4999
2,2,9,1575,2002,136731,1,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,38,71,1,1,13900
3,3,16,758,2017,19500,3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,29,14,1,1,45000
4,4,36,1077,2021,7388,3,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,29,10,1,1,97500


## Other feature extraction

In [13]:
def vehicle_age(df):
    current_year = 2024

    df['vehicle_age'] = current_year - df['model_year']
    df = df.drop(columns=['model_year'])

    return df

data = vehicle_age(data)
data_test = vehicle_age(data_test)

In [14]:
def engine_info(engine_total):
    HP_str = re.search(r'(\d+\.?\d*)HP', engine_total)
    size_str = re.search(r'(\d+\.?\d*)L', engine_total)
    cylinder_str = re.search(r'(\d+) Cylinder', engine_total)

    HP = float(HP_str.group(1)) if HP_str else None
    size = float(size_str.group(1)) if size_str else None
    cylinder = int(cylinder_str.group(1)) if cylinder_str else None

    return HP, size, cylinder

data[['HP', 'engine_size', 'cylinders']] = data['engine'].apply(lambda x: pd.Series(engine_info(x)))
data_test[['HP', 'engine_size', 'cylinders']] = data_test['engine'].apply(lambda x: pd.Series(engine_info(x)))

data = data.drop(columns=['engine'])
data_test = data_test.drop(columns=['engine'])

data.head()

,id,brand,model,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,vehicle_age,HP,engine_size,cylinders
0,0,31,495,213000,3,A/T,312,71,1,1,4200,17,172.0,1.6,4.0
1,1,28,930,143250,3,A/T,263,10,0,1,4999,22,252.0,3.9,8.0
2,2,9,1575,136731,1,A/T,38,71,1,1,13900,22,320.0,5.3,8.0
3,3,16,758,19500,3,Transmission w/Dual Shift Mode,29,14,1,1,45000,7,420.0,5.0,8.0
4,4,36,1077,7388,3,7-Speed A/T,29,10,1,1,97500,3,208.0,2.0,4.0


In [15]:
data_transmission = pd.DataFrame(data['transmission'].unique().tolist(), columns=['Transmission'])

# Feature 1: Extract the number of gears
data_transmission['speed'] = data_transmission['Transmission'].apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else None)

# Feature 2: Identify transmission type (automatic, manual, CVT)
data_transmission['transmission_type'] = data_transmission['Transmission'].apply(lambda x: 'Automatic' if 'A/T' in x or 'Automatic' in x or 'CVT' in x else 'Manual' if 'M/T' in x or 'Manual' in x else 'Other')

# # Feature 3: Special modes
# df_transmission['Dual_Shift_Mode'] = df_transmission['Transmission'].apply(lambda x: 1 if 'Dual Shift Mode' in x else 0)
# df_transmission['Auto_Shift'] = df_transmission['Transmission'].apply(lambda x: 1 if 'Auto-Shift' in x else 0)

data = pd.concat([data, data_transmission[['speed', 'transmission_type']]], axis=1)


data_test_transmission = pd.DataFrame(data_test['transmission'].unique().tolist(), columns=['Transmission'])

# Feature 1: Extract the number of gears
data_test_transmission['speed'] = data_test_transmission['Transmission'].apply(lambda x: re.search(r'\d+', x).group() if re.search(r'\d+', x) else None)

# Feature 2: Identify transmission type (automatic, manual, CVT)
data_test_transmission['transmission_type'] = data_test_transmission['Transmission'].apply(lambda x: 'Automatic' if 'A/T' in x or 'Automatic' in x or 'CVT' in x else 'Manual' if 'M/T' in x or 'Manual' in x else 'Other')

# # Feature 3: Special modes
# data_test_transmission['Dual_Shift_Mode'] = data_test_transmission['Transmission'].apply(lambda x: 1 if 'Dual Shift Mode' in x else 0)
# data_test_transmission['Auto_Shift'] = data_test_transmission['Transmission'].apply(lambda x: 1 if 'Auto-Shift' in x else 0)

data_test = pd.concat([data_test, data_test_transmission[['speed', 'transmission_type']]], axis=1)

In [16]:
print('HP pre processing:', data['HP'].unique())
data['HP'] = data['HP'].fillna('Unknown')
print('HP processing', data['HP'].unique())

print()

print('Engine size pre processing:', data['engine_size'].unique())
data['engine_size'] = data['engine_size'].fillna('Unknown')
print('Engine size processing', data['engine_size'].unique())

print()

print('Cylinders pre processing:', data['cylinders'].unique())
data['cylinders'] = data['cylinders'].fillna('Unknown')
print('Cylinders processing', data['cylinders'].unique())

print()

print('Speed pre processing:', data['speed'].unique())
data['speed'] = data['speed'].fillna('Unknown')
print('Speed post processing', data['speed'].unique())

print()

print('Transmission type pre processing:', data['transmission_type'].unique())
data['transmission_type'] = data['transmission_type'].fillna('Unknown')
print('Transmission type post processing', data['transmission_type'].unique())


data_test['HP'] = data_test['HP'].fillna('Unknown')
data_test['engine_size'] = data_test['engine_size'].fillna('Unknown')
data_test['cylinders'] = data_test['cylinders'].fillna('Unknown')
data_test['speed'] = data_test['speed'].fillna('Unknown')
data_test['transmission_type'] = data_test['transmission_type'].fillna('Unknown')

HP pre processing: [ 172.  252.  320.  420.  208.  333.  355.   nan  425.  312.  523.  503.
  395.  310.  469.  429.  210.  335.  316.  381.  450.  236.  211.  220.
  255.  518.  268.  174.  240.  248.  563.  241.  230.  430.  835.  300.
  280.  295.  348.  560.  285.  302.  490.  200.  416.  179.  385.  160.
  270.  350.  190.  181.  265.  345.  400.  261.  536.  402.  382.  180.
  617.  173.  389.  318.  325.  365.  340.  235.  169.  710.  225.  366.
  284.  260.  670.  480.  440.  301.  380.  616.  317.  435.  305.  471.
  443.  328.  550.  201.  276.  582.  290.  362.  184.  120.  275.  510.
  414.  375.  455.  520.  306.  292.  403.  454.  496.  166.  288.  640.
  293.  147.  263.  170.  577.  449.  296.  291.  467.  354.  283.  603.
  386.  264.  204.  215.  412.  150.  453.  140.  707.  242.  445.  410.
  500.  188.  557.  142.  199.  315.  282.  247.  182.  605.  161.  475.
  202.  349.  152.  228.  154.  383.  526.  760.  630.  212.  287.  717.
  521.  232.  444.  641.  308.  

In [17]:
data.replace('Unknown', 0, inplace=True)
data = data.drop(columns=['transmission_type', 'transmission', 'id'])
data = data.apply(pd.to_numeric, errors='coerce')

data_test.replace('Unknown', 0, inplace=True)
data_test = data_test.drop(columns=['transmission_type', 'transmission', 'id'])
data_test = data_test.apply(pd.to_numeric, errors='coerce')

C:\Users\tancr\AppData\Local\Temp\ipykernel_15344\2953739124.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace('Unknown', 0, inplace=True)
C:\Users\tancr\AppData\Local\Temp\ipykernel_15344\2953739124.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_test.replace('Unknown', 0, inplace=True)


## Create X_train and y_train

In [18]:
X_train = data.drop(columns=['price'])
y_train = data['price']

X_test = data_test

## Create and evaluate a model

In [25]:
from sklearn.ensemble import RandomForestRegressor

rf_regr = RandomForestRegressor(n_estimators=100)
rf_regr.fit(X_train, y_train)

y_pred1 = rf_regr.predict(X_train)
rmse1 = root_mean_squared_error(y_pred1, y_train)

print(rmse1)

29381.939427062236


In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf_regr = RandomForestRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100], # [50, 100, 200],   # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    #'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    #'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at a leaf node
    #'bootstrap': [True, False]        # Whether bootstrap samples are used when building trees
}

# Initialize GridSearchCV with RandomForestRegressor and param_grid
grid_search = GridSearchCV(estimator=rf_regr, param_grid=param_grid,
                           cv=5, scoring='neg_root_mean_squared_error')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters found: {grid_search.best_params_}")

# Predict on the training set
y_pred_train = grid_search.predict(X_train)
rmse_train = root_mean_squared_error(y_pred_train, y_train)

# # Predict on the test set
# y_pred_test = grid_search.predict(X_test)
# rmse_test = np.sqrt(root_mean_squared_error(y_test, y_pred_test))

# Print RMSE for both training and test sets
print(f"RMSE on training set: {rmse_train:.2f}")
# print(f"RMSE on test set: {rmse_test:.2f}")

Best parameters found: {'max_depth': 10, 'n_estimators': 100}
RMSE on training set: 58260.43


In [31]:
rf_regr = RandomForestRegressor(n_estimators=50) #, max_depth=10, min_samples_split=10)
rf_regr.fit(X_train, y_train)

y_pred1 = rf_regr.predict(X_train)
rmse1 = root_mean_squared_error(y_pred1, y_train)

print(rmse1)

29935.408484358337
